In [2]:
# import libraries
import json
import pandas as pd
import numpy as np

In [17]:
# load data

price_vol_usd_btc = pd.read_csv('data/bitcoinity_price_volume_USD_btc_5y.csv')    # Preis und Volumen in USD
google_trends = pd.read_csv('data/googleTrends_5y.csv')

# convert date to datetime
google_trends['date'] = pd.to_datetime(google_trends['Woche'])
price_vol_usd_btc['date'] = pd.to_datetime(price_vol_usd_btc['Time'])

# add a new column with year and week
google_trends['year_week'] = google_trends['date'].dt.strftime('%Y-%U')
price_vol_usd_btc['year_week'] = price_vol_usd_btc['date'].dt.strftime('%Y-%U')

# merge data by year and week
weekly_data = pd.merge(google_trends, price_vol_usd_btc, on='year_week')

weekly_data.drop(columns=['Woche', 'date_x', 'date_y', 'Time'], inplace=True)
weekly_data.rename(columns={'volume': 'volume_usd'}, inplace=True)

weekly_data.set_index('year_week', inplace=True)

In [18]:
# build up the final data structure

for col in weekly_data.columns:
    if col != 'price':
        weekly_data[f'{col}-1w'] = weekly_data[col].shift(1)
        weekly_data[f'{col}-2w'] = weekly_data[col].shift(2)
        weekly_data[f'{col}-5w'] = weekly_data[col].shift(5)

weekly_data['price+1w'] = weekly_data['price'].shift(-1)



In [4]:


with open('data/blockchain_tx-per-day_tx-vol.json', 'r') as f:
    tx_vol = json.load(f)

with open('data/blockchain_tx-per_day_tx-vol-usd.json', 'r') as f:
    tx_usd = json.load(f)

with open('data/blockchain_trade-volume_market-price.json', 'r') as f:
    vol_price = json.load(f)

   

In [9]:
# estimated-transaction-volume per day in btc
df_tx_vol = pd.DataFrame(tx_vol['estimated-transaction-volume'])
df_tx_vol['time'] = pd.to_datetime(df_tx_vol['x'], unit='ms')
df_tx_vol.set_index('time', inplace=True)
df_tx_vol.drop(columns='x', inplace=True)
df_tx_vol.rename(columns={'y': 'estimated_tx_vol_usd'}, inplace=True)


# number of transactions per day
df_n_tx = pd.DataFrame(tx_vol['n-transactions'])
df_n_tx['time'] = pd.to_datetime(df_n_tx['x'], unit='ms')
df_n_tx.set_index('time', inplace=True)
df_n_tx.drop(columns='x', inplace=True)
df_n_tx.rename(columns={'y': 'n-transactions'}, inplace=True)

# estimated-transaction-volume per day in usd
df_tx_vol_usd = pd.DataFrame(tx_usd['estimated-transaction-volume-usd'])
df_tx_vol_usd['time'] = pd.to_datetime(df_tx_vol_usd['x'], unit='ms')
df_tx_vol_usd.set_index('time', inplace=True)
df_tx_vol_usd.drop(columns='x', inplace=True)
df_tx_vol_usd.rename(columns={'y': 'estimated-transaction-volume-usd'}, inplace=True)




# daily trade volume in btc
df_trade_vol = pd.DataFrame(vol_price['trade-volume'])
df_trade_vol['time'] = pd.to_datetime(df_trade_vol['x'], unit='ms')
df_trade_vol.set_index('time', inplace=True)
df_trade_vol.drop(columns='x', inplace=True)
df_trade_vol.rename(columns={'y': 'trade-volume'}, inplace=True)

# market price in usd
df_market_price = pd.DataFrame(vol_price['market-price'])
df_market_price['time'] = pd.to_datetime(df_market_price['x'], unit='ms')
df_market_price.set_index('time', inplace=True)
df_market_price.drop(columns='x', inplace=True)
df_market_price.rename(columns={'y': 'market-price'}, inplace=True)


# combine all dataframes
df = pd.concat([df_tx_vol, df_tx_vol_usd, df_n_tx, df_trade_vol, df_market_price], axis=1)

df = df[df.index > '2019-12-28']

# create a new index colum from the time colum in the format yyyy-week
df['week'] = df.index.strftime('%Y-%U')



